In [1]:
import tensorflow as tf 
from tensorflow import keras 
from keras import layers

Download and prepare dataset

In [ ]:
vocab_size = 20000 # Only consider the top 20k words
num_tokens_per_example = 200 #  Only consider the first 200 words of each movie review
(x_train , y_train) ,(x_val , y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), 'Training sequences')
print(len(x_val), "Validation sequences")

x_train = keras.utils.pad_sequences(
    x_train , maxlen=num_tokens_per_example
)
x_val = keras.utils.pad_sequences(x_val , maxlen=num_tokens_per_example)

Define hyper parameters

In [ ]:
embed_dim = 32 # Embedding size for each token 
num_heads = 2 # Number of attention heads 
ff_dim = 32 # Hidden layer size in feedforward network.
num_experts = 10 # Number of experts used in the Switch Transformer.
batch_size = 50  # Batch size 
learning_rate = 0.001
dropout_rate = 0.25 
num_epochs = 3 # Number of epochs 
num_tokens_per_batch = (
    batch_size * num_tokens_per_example
) # Total number of tokens per batch
print(f'Number of tokens per batch: {num_tokens_per_batch}')

Implement token & position embedding layer 

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

Implement the feedforward network 

In [ ]:
def create_feedforward_network(ff_dim, name=None):
    return keras.Sequential(
        [layers.Dense(ff_dim, activation="relu"), layers.Dense(ff_dim)], name=name
    )

Implement the load-balanced loss 

In [ ]:
def load_balanced_loss(router_probs ,expert_mask):
    # Router_probs [token_per_batch , num_experts] là xác xuất được chỉ định cho mỗi chuyên gia 
    # vơí mỗi mã thông báo 
    # Expert_mask [ token_per_batch , num_experts] chứa chuyên gia có xác xuất bộ định tuyến cao nhất 
    # ở định dạng one-hot 
    num_experts = tf.shape(expert_mask)[-1]
    # Lấy số lượng chuyên gia từ kích thước của ma trận expert_mask, 
    # là một ma trận nhị phân có giá trị 1 nếu token được gửi đến chuyên gia tương ứng và 0 nếu không.
    density = tf.reduce_mean(expert_mask , axis=0)
    # Tính tỷ lệ token được gửi đến mỗi chuyên gia, bằng cách lấy trung bình theo trục 0 của ma trận expert_mask. 
    # Kết quả là một vector density có độ dài bằng số lượng chuyên gia và tổng bằng 1.
    density_proxy = tf.reduce_mean(router_probs , axis=0)
    # Tính tỷ lệ xác suất được gán cho mỗi chuyên gia từ bộ định tuyến, 
    # bằng cách lấy trung bình theo trục 0 của ma trận router_probs, 
    # là một ma trận có giá trị từ 0 đến 1 cho biết xác suất của token được gửi đến chuyên gia tương ứng. 
    # Kết quả là một vector density_proxy có độ dài bằng số lượng chuyên gia và tổng bằng 1.
    loss = tf.reduce_mean(density_proxy * density) * tf.cast(
        (num_experts ** 2), tf.dtypes.float32
    )
    # Tính hàm mất mát bằng cách lấy tích vô hướng của hai vector density và density_proxy, 
    # rồi nhân với bình phương của số lượng chuyên gia. Hàm mất mát này sẽ có giá trị nhỏ nhất khi hai vector này là đồng nhất, 
    # tức là khi cả hai đều có giá trị bằng 1/số lượng chuyên gia. 
    # Điều này có nghĩa là token và xác suất được phân phối đều cho các chuyên gia.
    return loss


In [ ]:
class Router(layers.Layer):
    def __init__(self, num_experts, expert_capacity):
        self.num_experts = num_experts
        self.route = layers.Dense(units=num_experts)
        self.expert_capacity = expert_capacity
        super().__init__()

    def call(self, inputs, training=False):
        # inputs shape: [tokens_per_batch, embed_dim]
        # router_logits shape: [tokens_per_batch, num_experts]
        router_logits = self.route(inputs)

        if training:
            # Nếu đang ở chế độ huấn luyện, thêm nhiễu ngẫu nhiên vào router_logits để khuyến khích việc khám phá các chuyên gia khác nhau. 
            # Nhiễu ngẫu nhiên có giá trị từ 0.9 đến 1.1.
            router_logits += tf.random.uniform(
                shape=router_logits.shape, minval=0.9, maxval=1.1
            )
        # Tính router_probs là một tensor có kích thước [tokens_per_batch, num_experts] chứa xác suất của mỗi token được gửi đến mỗi chuyên gia. 
        # Xác suất được tính bằng hàm softmax trên trục -1 của router_logits
        router_probs = keras.activations.softmax(router_logits, axis=-1)
        # Lấy chuyên gia có xác suất cao nhất cho mỗi token. 
        # expert_gate là một tensor có kích thước [tokens_per_batch, 1] chứa xác suất cao nhất từ router_probs cho mỗi token. 
        # expert_index là một tensor có kích thước [tokens_per_batch, 1] chứa chỉ số của chuyên gia tương ứng với xác suất cao nhất cho mỗi token.
        expert_gate, expert_index = tf.math.top_k(router_probs, k=1)
        # Tính expert_mask là một tensor có kích thước [tokens_per_batch, num_experts] chứa giá trị nhị phân cho biết token nào được gửi đến chuyên gia nào. 
        # Giá trị này được tính bằng cách sử dụng hàm tf.one_hot với expert_index và depth là num_experts.
        expert_mask = tf.one_hot(expert_index, depth=self.num_experts)
        # Tính hàm mất mát cân bằng tải bằng cách gọi hàm load_balanced_loss với router_probs và expert_mask làm đầu vào. 
        # Thêm hàm mất mát này vào danh sách các hàm mất mát của lớp bằng phương thức self.add_loss.
        aux_loss = load_balanced_loss(router_probs, expert_mask)
        self.add_loss(aux_loss)
        # Tính position_in_expert là một tensor có kích thước [tokens_per_batch, num_experts] cho biết vị trí của token trong hàng đợi của mỗi chuyên gia. 
        # Giá trị này được tính bằng cách lấy tổng tích lũy theo trục 0 của expert_mask rồi nhân với expert_mask. 
        # Sau đó, ép kiểu tensor này sang kiểu int32.
        position_in_expert = tf.cast(
            tf.math.cumsum(expert_mask, axis=0) * expert_mask, tf.dtypes.int32
        )
        # Lọc ra các token có vị trí trong hàng đợi của chuyên gia nhỏ hơn expert_capacity, tức là các token không vượt quá khả năng xử lý của chuyên gia. 
        # Điều này được thực hiện bằng cách sử dụng hàm tf.math.less để so sánh position_in_expert và expert_capacity, 
        # rồi ép kết quả sang kiểu float32 và nhân với expert_mask. Kết quả là expert_mask được cập nhật lại để loại bỏ các token không được gửi đến các chuyên gia.
        expert_mask *= tf.cast(
            tf.math.less(
                tf.cast(position_in_expert, tf.dtypes.int32), self.expert_capacity
            ),
            tf.dtypes.float32,
        )
        # Tính expert_mask_flat là một tensor có kích thước [tokens_per_batch] bằng cách lấy tổng theo trục -1 của expert_mask. 
        # Giá trị này cho biết token nào được gửi đến ít nhất một chuyên gia.
        expert_mask_flat = tf.reduce_sum(expert_mask, axis=-1)

        # Cập nhật lại expert_gate bằng cách nhân với expert_mask_flat để loại bỏ các token không được gửi đến bất kỳ chuyên gia nào.
        expert_gate *= expert_mask_flat
        #  Giá trị này cho biết xác suất định tuyến và hệ số cân bằng tải của mỗi token đối với mỗi chuyên gia và mỗi vị trí trong hàng đợi của chuyên gia.
        combined_tensor = tf.expand_dims(
            expert_gate
            * expert_mask_flat
            * tf.squeeze(tf.one_hot(expert_index, depth=self.num_experts), 1),
            -1,
        ) * tf.squeeze(tf.one_hot(position_in_expert, depth=self.expert_capacity), 1)
        # Tính dispatch_tensor là một tensor có kích thước [tokens_per_batch, num_experts, expert_capacity] bằng cách ép kiểu combined_tensor sang kiểu float32. 
        # Giá trị này cho biết token nào được gửi đến chuyên gia nào và vị trí nào trong hàng đợi của chuyên gia bằng giá trị nhị phân 0 hoặc 1.
        dispatch_tensor = tf.cast(combined_tensor, tf.dtypes.float32)

        return dispatch_tensor, combined_tensor

In [ ]:
class Switch(layers.Layer):
    def __init__(self, num_experts, embed_dim, num_tokens_per_batch, capacity_factor=1):
        self.num_experts = num_experts
        self.embed_dim = embed_dim
        self.experts = [
            create_feedforward_network(embed_dim) for _ in range(num_experts)
        ]

        self.expert_capacity = num_tokens_per_batch // self.num_experts
        self.router = Router(self.num_experts, self.expert_capacity)
        super().__init__()

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        num_tokens_per_example = tf.shape(inputs)[1]

        # inputs shape : [num_token_per_patch , embed_dim]
        inputs = tf.reshape(inputs, [num_tokens_per_batch, self.embed_dim])
        # dispatch_tensor (tensor gửi đi ) shape : [expert_capacity , num_experts, tokens_per_batch]
        # combine_tensor (tensor kết hợp) shape : [token_per_batch , num_experts , expert_capacity]
        dispatch_tensor, combine_tensor = self.router(inputs)
        # expert_inputs shape : [num_experts, expert_capacity , embed_dim]
        # tính toán một tensor mới có kích thước là [expert_capacity, num_experts, embed_dim] 
        # bằng cách thực hiện phép nhân ma trận giữa tensor inputs và tensor dispatch_tensor theo công thức ‘ab,acd->cdb’
        expert_inputs = tf.einsum("ab,acd->cdb", inputs, dispatch_tensor)
        expert_inputs = tf.reshape(
            expert_inputs, [self.num_experts, self.expert_capacity, self.embed_dim]
        )
        # Dispatch to experts (gửi đến chuyên gia)
        # dùng hàm tf.unstack để tách tensor expert_inputs thành một danh sách các tensor có kích thước là [expert_capacity, embed_dim] theo chiều thứ nhất (num_experts). 
        # Hàm tf.unstack cho phép bạn tách một tensor có kích thước là R thành một danh sách các tensor có kích thước là R-1 theo một chiều nào đó
        expert_input_list = tf.unstack(expert_inputs, axis=0)
        expert_output_list = [
            self.experts[idx](expert_input)
            for idx, expert_input in enumerate(expert_input_list)
        ]
        # Expert_outputs shape : [expert_capacity , num_expert , embeb_dim ]
        expert_outputs = tf.stack(expert_output_list, axis=1)
        # expert_outputs_combined shape: [tokens_per_batch, embed_dim]
        expert_outputs_combined = tf.einsum(
            "abc,xba->xc", expert_outputs, combine_tensor
        )
        # output_shape : [batch_size , num_tokens_per_example , embed_dim]
        outputs = tf.reshape(
            expert_outputs_combined,
            [batch_size, num_tokens_per_example, self.embed_dim],
        )
        return outputs
    


Implement a Transformer block layer

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ffn, dropout_rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # The ffn can be either a standard feedforward network or a switch
        # layer with a Mixture of Experts.
        self.ffn = ffn
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Implement the classifier

In [ ]:
def create_classifier():
    switch = Switch(num_experts, embed_dim, num_tokens_per_batch)
    transformer_block = TransformerBlock(ff_dim, num_heads, switch)

    inputs = layers.Input(shape=(num_tokens_per_example,))
    embedding_layer = TokenAndPositionEmbedding(
        num_tokens_per_example, vocab_size, embed_dim
    )
    x = embedding_layer(inputs)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(2, activation="softmax")(x)

    classifier = keras.Model(inputs=inputs, outputs=outputs)
    return classifier

Train and evaluate the model

In [ ]:
def run_experiment(classifier):
    classifier.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    history = classifier.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=(x_val, y_val),
    )
    return history


classifier = create_classifier()
run_experiment(classifier)